In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
#!pip install opencv-python
#!pip install pyproj

# Helper: fit a 2D line via PCA
def fit_line_2d(pts: np.ndarray):
    pca = PCA(n_components=1)
    pca.fit(pts)
    direction = pca.components_[0]
    centroid = pca.mean_
    direction /= np.linalg.norm(direction)
    return centroid, direction

# Helper: angle between a direction vector and the image horizontal axis
def angle_to_horizontal(direction: np.ndarray):
    horiz = np.array([1.0, 0.0])
    cosang = np.clip(np.dot(direction, horiz), -1.0, 1.0)
    return np.degrees(np.arccos(abs(cosang)))

# Main pipeline function
def process_images(df: pd.DataFrame,
                   image_width_px: int,
                   image_height_px: int,
                   barcode_vis_thresh: float = 0.7):
    """
    Processes a DataFrame of detected objects to produce, per left-to-right rank across images,
    the pallet points and corresponding camera GPS.

    Parameters:
      df: pandas.DataFrame with columns:
        ['image', 'class', 'x1', 'x2', 'y1', 'y2', 'latitude', 'longitude', 'altitude']
        where 'class' is either 'barcode' or 'pallets'.
      image_width_px: width of the image in pixels (for optical center X coordinate).
      image_height_px: height of the image in pixels (for optical center Y coordinate).
      barcode_vis_thresh: fraction of the maximum barcodes (per any image) required to process an image.

    Returns:
      final_list: list of dicts, one per left-to-right rank index.
        Each dict maps image name to a sub-dict with:
          - '<ordinal> point': (x, y) pixel coordinate of the pallet point.
          - 'lat', 'lon', 'alt': camera position for that image.
    """
    # 1) Determine threshold count from maximum barcodes in any image
    bc_counts = df[df['class']=='barcode'].groupby('image').size()
    max_barcodes = int(bc_counts.max()) if not bc_counts.empty else 0
    thresh_count = max_barcodes * barcode_vis_thresh

    # Precompute optical center of the image
    optical_center = np.array([image_width_px / 2.0, image_height_px / 2.0], dtype=float)

    # 2) Fit and select pallet points per image
    sorted_pts = {}    # maps image -> list of sorted (x,y) points
    cam_meta   = {}   # maps image -> {'lat', 'lon', 'alt'}

    for img, grp in df.groupby('image'):
        # a) Check barcode visibility
        count_barcode = (grp['class']=='barcode').sum()
        if count_barcode < thresh_count:
            continue

        # b) Record camera GPS metadata
        lat = grp['latitude'].iloc[0]
        lon = grp['longitude'].iloc[0]
        alt = grp['altitude'].iloc[0]
        cam_meta[img] = {'lat': lat, 'lon': lon, 'alt': alt}

        # c) Extract pallet coordinates
        pal = grp[grp['class']=='pallets']
        if pal.shape[0] < 2:
            continue
        pts1 = pal[['x1','y1']].to_numpy()
        pts2 = pal[['x2','y2']].to_numpy()

        # d) Fit two lines and require horizontal orientation
        _, dir1 = fit_line_2d(pts1)
        if angle_to_horizontal(dir1) >= 45.0:
            continue
        # (optional) could check parallelism with second line

        # e) Choose the line closer to the optical center
        d1 = np.linalg.norm(pts1 - optical_center, axis=1).mean()
        d2 = np.linalg.norm(pts2 - optical_center, axis=1).mean()
        chosen = pts1 if d1 < d2 else pts2

        # f) Sort chosen points left-to-right
        order = np.argsort(chosen[:,0])
        sorted_pts[img] = chosen[order].tolist()

    # 3) Build final_list by left-to-right rank
    final_list = []
    if sorted_pts:
        max_rank = max(len(pts) for pts in sorted_pts.values())
        for rank in range(max_rank):
            rank_entry = {}
            for img, pts in sorted_pts.items():
                if rank >= len(pts):
                    continue
                x, y = pts[rank]

                # Assemble entry using stored cam_meta
                meta = cam_meta[img]
                rank_entry[img] = {
                     "point": (x, y),
                    'lat': meta['lat'],
                    'lon': meta['lon'],
                    'alt': meta['alt']
                }
            final_list.append(rank_entry)

    return final_list

import nbimporter
from add_camera_position import get_df_with_camera_position
df = get_df_with_camera_position()

result = process_images(df, image_width_px=4032, image_height_px=3024)



import numpy as np
import cv2
from pyproj import Transformer
from scipy.spatial.transform import Rotation as R

def get_3d(data):
    """
    data: dict mapping image_filename -> {
      'leftmost point': (x_px, y_px),
      'lat': float, 'lon': float, 'alt': float
    }
    """

    # 1) Camera intrinsics (DJI M4TD wide) – unchanged
    sensor_width_mm  = 7.6
    sensor_height_mm = 5.7
    focal_length_mm  = 7.0
    image_width_px   = 1280
    image_height_px  = 720

    fx = (focal_length_mm / sensor_width_mm)  * image_width_px
    fy = (focal_length_mm / sensor_height_mm) * image_height_px
    cx, cy = image_width_px/2, image_height_px/2

    K = np.array([[fx,  0, cx],
                  [ 0, fy, cy],
                  [ 0,  0,  1]])

    # 2) CRS transformers
    to_ecef   = Transformer.from_crs("epsg:4326","epsg:4978",always_xy=True)
    from_ecef = Transformer.from_crs("epsg:4978","epsg:4326",always_xy=True)

    # 3) Nadir rotation
    rot_nadir = R.from_euler('xyz', [0, -90, 0], degrees=True).as_matrix()

    # 4) Build projection matrices
    P_list = []
    for img_name in data:
        values= data[img_name]
        Cx, Cy, Cz = to_ecef.transform(values["lon"], values["lat"], values["alt"])
        C = np.array([[Cx],[Cy],[Cz]])
        Rt = np.hstack((rot_nadir, -rot_nadir @ C))
        P_list.append(K @ Rt)

    # 5) Collect 2D points
    pts = np.array([data[img]["point"] for img in data], dtype=float)
    if pts.shape[0] < 2:
        raise ValueError("Need at least two cameras/points to triangulate")

    p1 = pts[0].reshape(2,1)
    p2 = pts[1].reshape(2,1)

    # 6) Triangulate
    Xh = cv2.triangulatePoints(P_list[0], P_list[1], p1, p2)

    # 7) From homogeneous to ECEF
    X_ecef = (Xh[:3] / Xh[3]).flatten()

    # 8) Back to GPS
    lon, lat, alt = from_ecef.transform(*X_ecef)

    print(f"Triangulated GPS coordinates: lat={lat:.6f}, lon={lon:.6f}, alt≈{alt:.2f} m")
    return lon, lat, alt


points_3d = []
for coord in result:
    print(coord)
    
    points_3d.append(get_3d(coord))

print("FINAL", points_3d)

